In [1]:
import os
import glob
from PIL import Image

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output

In [ ]:
image_folder= '/Users/rfalcao/Documents/Cusp Images - SEM'
image_files = sorted(os.listdir(image_folder))
image_path = os.path.join(image_folder, image_files[0])

In [2]:
from Rotation import ImageAlignment 

In [5]:
aligner = ImageAlignment('/Users/rfalcao/Documents/Clear Cusps', '/Users/rfalcao/Documents/ClearCuspsCroppedRotated',False)
aligner.process_all_images(method="translation") #Methods: 'fft' , 'radon', 'translation' , 'crop_only' , 'all

Processing Images (translation):   0%|          | 0/44 [00:00<?, ?it/s]

Processing Images (translation): 100%|██████████| 44/44 [04:05<00:00,  5.57s/it]

Rotation Angles: {'12.tif': -3, '8 (1).tif': 87, '2b_i_i_2_1000x.tif': -48, 'FM024_3_a_i_AOI4_5000x.tif': -2, '11.tif': -25, '2b_iii_ii_5_1000x.tif': 36, 'RT_dry_1mmmin_500x.tif': -89, '5 (2).tif': 12, '15.tif': -5, '90C_dry_0.1mmmin_500x.tif': -89, '6 (4).tif': -1, 'FM024_3_a_i_AOI4_1500x.tif': -3, '90C_dry_100mmmin_500x.tif': 3, '-55C_dry_1mmmin_1000x.tif': -88, '-55C_wet_1mmmin_500x.tif': 2, 'FM024_3_a_i_AOI8_1300x.tif': 6, 'RT_500x.tif': -89, '11 (1).tif': -2, '6 (3).tif': -8, '90C_dry_100mmmin_1000x.tif': 2, '2 (1).tif': -85, '2bi_iii_4_1000x.tif': -64, 'FM024_1_d_i_flatbed_ML1_1_atTTR_2000x_45degtilt.tif': 3, 'FM024_3_e_i_flatbed_ML1_2_1000x_45degtilt.tif': 37, '6 (1).tif': 12, '11 (3).tif': 1, 'FM024_1_b_ii_delam2_ML1_1_1300x.tif': 21, 'FM024_1_b_ii_delam2_ML1_3_400x.tif': -8, '1.tif': -26, '23 (1).tif': -6, '5.tif': -89, '11 (2).tif': -6, '90C_dry_0.1mmmin_1000x.tif': 87, '2b_i_ii_2_2000x.tif': -88, '26.tif': -6, '25.tif': -6, '24.tif': -5, '18.tif': 8, '2b_iii_i_3_2000x.tif': 

{'12.tif': -3,
 '8 (1).tif': 87,
 '2b_i_i_2_1000x.tif': -48,
 'FM024_3_a_i_AOI4_5000x.tif': -2,
 '11.tif': -25,
 '2b_iii_ii_5_1000x.tif': 36,
 'RT_dry_1mmmin_500x.tif': -89,
 '5 (2).tif': 12,
 '15.tif': -5,
 '90C_dry_0.1mmmin_500x.tif': -89,
 '6 (4).tif': -1,
 'FM024_3_a_i_AOI4_1500x.tif': -3,
 '90C_dry_100mmmin_500x.tif': 3,
 '-55C_dry_1mmmin_1000x.tif': -88,
 '-55C_wet_1mmmin_500x.tif': 2,
 'FM024_3_a_i_AOI8_1300x.tif': 6,
 'RT_500x.tif': -89,
 '11 (1).tif': -2,
 '6 (3).tif': -8,
 '90C_dry_100mmmin_1000x.tif': 2,
 '2 (1).tif': -85,
 '2bi_iii_4_1000x.tif': -64,
 'FM024_1_d_i_flatbed_ML1_1_atTTR_2000x_45degtilt.tif': 3,
 'FM024_3_e_i_flatbed_ML1_2_1000x_45degtilt.tif': 37,
 '6 (1).tif': 12,
 '11 (3).tif': 1,
 'FM024_1_b_ii_delam2_ML1_1_1300x.tif': 21,
 'FM024_1_b_ii_delam2_ML1_3_400x.tif': -8,
 '1.tif': -26,
 '23 (1).tif': -6,
 '5.tif': -89,
 '11 (2).tif': -6,
 '90C_dry_0.1mmmin_1000x.tif': 87,
 '2b_i_ii_2_2000x.tif': -88,
 '26.tif': -6,
 '25.tif': -6,
 '24.tif': -5,
 '18.tif': 8,
 '2b

In [ ]:
import os
import cv2
import numpy as np
import shutil
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output

def classify_clear_cusp_images(image_folder, output_folder):
    """
    Iterates through images in a folder, displays them for manual classification
    using interactive widgets, and saves selected images with clear cusps.

    Parameters:
        - image_folder: Folder containing images to classify.
        - output_folder: Folder where selected images will be saved.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get list of image files
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.tif'))]
    image_index = [0]  # Mutable index to track current image

    output = Output()

    def update_display():
        """Updates the display to show the current image."""
        with output:
            clear_output(wait=True)
            if image_index[0] >= len(image_files):
                print("Classification complete! Selected images have been saved.")
                return
            
            # Load and display the current image
            filename = image_files[image_index[0]]
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            plt.figure(figsize=(6, 6))
            plt.title(f"Does this image have clear cusps? \nFilename: {filename}")
            plt.imshow(image, cmap='gray')
            plt.axis('off')
            plt.show()

    def on_save_clicked(b):
        """Handles 'Save' button click (move image to output folder)."""
        filename = image_files[image_index[0]]
        src_path = os.path.join(image_folder, filename)
        dst_path = os.path.join(output_folder, filename)
        shutil.copy(src_path, dst_path)  # Copy the image to the new folder
        next_image()

    def on_skip_clicked(b):
        """Handles 'Skip' button click."""
        next_image()

    def next_image():
        """Moves to the next image or ends classification if all are processed."""
        image_index[0] += 1
        update_display()

    # Create buttons
    save_button = Button(description="Yes", button_style='success', layout=Layout(width='100px'))
    skip_button = Button(description="No", button_style='warning', layout=Layout(width='100px'))

    save_button.on_click(on_save_clicked)
    skip_button.on_click(on_skip_clicked)

    # Display UI
    controls = HBox([save_button, skip_button])
    display(VBox([output, controls]))

    update_display()

# Paths
image_folder = '/Users/rfalcao/Documents/Cusp Images - SEM'  # Replace with the actual folder
output_folder = '/Users/rfalcao/Documents/Clear Cusps'  # Replace with the actual folder

# Run the classification
classify_clear_cusp_images(image_folder, output_folder)


In [11]:
import os
import cv2
import numpy as np
import shutil
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output
#LABELING CODE
def label_cusps(image_folder):
    """
    Iterates through images in a folder, displays them for manual classification
    using interactive widgets, and saves selected images with clear cusps.

    Parameters:
        - image_folder: Folder containing images to classify.
        - output_folder: Folder where selected images will be saved.
    """
    

    # Get list of image files
    image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.tif'))]
    image_index = [0]  # Mutable index to track current image
    classifications={}
    output = Output()

    def update_display():
        """Updates the display to show the current image."""
        with output:
            clear_output(wait=True)
            if image_index[0] >= len(image_files):
                print("Labels Complete! Selected images have been saved.")
                return
            
            # Load and display the current image
            filename = image_files[image_index[0]]
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            plt.figure(figsize=(6, 6))
            plt.title(f"Does this image have clear cusps? \nFilename: {filename}")
            plt.imshow(image, cmap='gray')
            plt.axis('off')
            plt.show()

    def on_yes_clicked(b):
        """Handles 'Save' button click (move image to output folder)."""
        filename = image_files[image_index[0]]
        classifications[filename]=1
        next_image()

    def on_no_clicked(b):
        filename = image_files[image_index[0]]
        classifications[filename]=0
        next_image()

    def next_image():
        """Moves to the next image or ends classification if all are processed."""
        image_index[0] += 1
        update_display()

    # Create buttons
    yes_button = Button(description="Cusp", button_style='success', layout=Layout(width='100px'))
    no_button = Button(description="No Cusp", button_style='warning', layout=Layout(width='100px'))

    yes_button.on_click(on_yes_clicked)
    no_button.on_click(on_no_clicked)

    # Display UI
    controls = HBox([yes_button, no_button])
    display(VBox([output, controls]))

    update_display()
    return classifications
# Paths
image_folder = "/Users/rfalcao/Documents/Separated Clear Cusps" 
# Run the classification
labels=label_cusps(image_folder)


Segmentation code - DeepSeek

In [12]:
print(labels)

{'12_q3_q1_q3_q3.png': 1, '12_q3_q3_q2_q0.png': 1, '12_q3_q3_q2_q1.png': 1, '12_q3_q1_q3_q2.png': 1, '12_q3_q1_q3_q0.png': 1, '12_q3_q3_q2_q3.png': 0, '12_q3_q3_q2_q2.png': 1, '12_q3_q1_q3_q1.png': 1, '12_q3_q3_q0_q3.png': 1, '12_q3_q1_q1_q0.png': 1, '12_q3_q1_q1_q1.png': 1, '12_q3_q3_q0_q2.png': 1, '12_q3_q3_q0_q0.png': 1, '12_q3_q3_q0_q1.png': 1, '12_q2_q1_q3.png': 1, '12_q2_q1_q2.png': 1, '12_q3_q0_q3_q0.png': 0, '12_q2_q1_q0.png': 1, '12_q2_q1_q1.png': 1, '12_q3_q0_q3_q1.png': 1, '8 (1)_q3.png': 1, '11_q3_q0.png': 1, '11_q3_q1.png': 1, '8 (1)_q2.png': 1, '12_q2_q3_q1.png': 1, '8 (1)_q0.png': 1, '11_q3_q2.png': 1, '8 (1)_q1.png': 1, '12_q2_q3_q2.png': 1, '12_q3_q1_q2_q1.png': 1, '12_q3_q3_q3_q2.png': 0, '12_q3_q3_q3_q3.png': 0, '12_q1_q3.png': 1, '12_q3_q1_q2_q0.png': 1, '12_q3_q1_q2_q2.png': 1, '12_q3_q3_q3_q1.png': 1, '12_q1_q1_q0.png': 1, '12_q3_q3_q3_q0.png': 1, '12_q3_q1_q2_q3.png': 1, '12_q3_q3_q1_q1.png': 1, '12_q3_q3_q1_q0.png': 1, '12_q3_q3_q1_q2.png': 1, '12_q3_q1_q0_q1.pn

In [7]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, HBox, VBox, Output, Layout
from IPython.display import display, clear_output

input_folder = '/Users/rfalcao/Documents/ClearCuspsCroppedRotated'  
output_folder = "/Users/rfalcao/Documents/Separated Clear Cusps"  
os.makedirs(output_folder, exist_ok=True)

image_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.tif'))]
image_queue = image_files[:]  
quartered_queue = []  

output = Output()

def update_display(image, title):
    with output:
        clear_output(wait=True)
        plt.figure(figsize=(6, 6))
        plt.title(title)
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        plt.show()

# Function to quarter image
def quarter_image(image):
    h, w = image.shape
    half_h, half_w = h // 2, w // 2
    return [
        image[:half_h, :half_w], image[:half_h, half_w:],
        image[half_h:, :half_w], image[half_h:, half_w:]
    ]

# Function to process next image
def next_image():
    if quartered_queue:
        # Process the most recent quartered image first
        q_image, base_name, i = quartered_queue[-1]
        update_display(q_image, f"Quarter {i} of {base_name}")
    elif image_queue:
        # Process the next image in the original queue
        image_path = image_queue[0]
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        update_display(image, f"Inspecting: {os.path.basename(image_path)}")
    else:
        with output:
            clear_output(wait=True)
            print("All images processed!")

# Button callbacks
def on_yes_clicked(b):
    if quartered_queue:
        # Quarter the most recent quartered image
        q_image, base_name, i = quartered_queue.pop()
        if q_image.shape[0] > 50 and q_image.shape[1] > 50:  # Only split if large enough
            quarters = quarter_image(q_image)
            for j, q in enumerate(quarters):
                quartered_queue.append((q, f"{base_name}_q{i}", j))  # Add new quarters to the stack
    elif image_queue:
        # Quarter the next image in the original queue
        image_path = image_queue.pop(0)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image.shape[0] > 50 and image.shape[1] > 50:  # Only split if large enough
            quarters = quarter_image(image)
            base_name = os.path.splitext(os.path.basename(image_path))[0]
            for i, q in enumerate(quarters):
                quartered_queue.append((q, base_name, i))  # Add quarters to the stack
    next_image()  
def on_no_clicked(b):
    if quartered_queue:
        quartered_queue.pop() 
    next_image()  

def on_save_clicked(b):
    if quartered_queue:
        q_image, base_name, i = quartered_queue.pop()
        save_path = os.path.join(output_folder, f"{base_name}_q{i}.png")
        cv2.imwrite(save_path, q_image)  
        print(f"Saved: {save_path}")
    next_image()  

# Create buttons
yes_button = Button(description="Yes (Quarter)", button_style='success', layout=Layout(width='150px'))
no_button = Button(description="No (Skip)", button_style='warning', layout=Layout(width='150px'))
save_button = Button(description="Save (Final Cusp)", button_style='primary', layout=Layout(width='150px'))

yes_button.on_click(on_yes_clicked)
no_button.on_click(on_no_clicked)
save_button.on_click(on_save_clicked)

controls = HBox([yes_button, no_button, save_button])
display(VBox([output, controls]))

# Start processing
next_image()

Saved: /Users/rfalcao/Documents/Separated Clear Cusps/12_q3_q3_q3_q3.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/12_q3_q3_q3_q2.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/12_q3_q3_q3_q1.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/12_q3_q3_q3_q0.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/12_q3_q3_q2_q3.png


...

Saved: /Users/rfalcao/Documents/Separated Clear Cusps/11_q3_q0.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/11_q2_q3.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/11_q2_q2.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/11_q2_q1.png


Saved: /Users/rfalcao/Documents/Separated Clear Cusps/11_q2_q0.png


In [ ]:


input_folder = '/Users/rfalcao/Documents/ClearCuspsCroppedRotated'  
output_folder = "/Users/rfalcao/Documents/Separated Clear Cusps"  
os.makedirs(output_folder, exist_ok=True)

# Load images
image_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.tif'))]
image_queue = image_files[:]  # Initialize queue with original images

output = Output()

# Global variables for selected region
selected_region = None

# Function to display image
def update_display(image, title):
    global selected_region
    selected_region = None  # Reset selected region
    with output:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10, 10))  # Larger plot size
        ax.imshow(image, cmap='gray')
        ax.set_title(title)
        ax.axis('off')

        # Callback function for RectangleSelector
        def on_select(eclick, erelease):
            global selected_region
            x1, y1 = int(eclick.xdata), int(eclick.ydata)
            x2, y2 = int(erelease.xdata), int(erelease.ydata)
            selected_region = (min(x1, x2), min(y1, y2), abs(x2 - x1), abs(y2 - y1))  # (x, y, width, height)
            print(f"Selected region: {selected_region}")

        # Add RectangleSelector
        rs = RectangleSelector(ax, on_select, button=[1], minspanx=5, minspany=5, spancoords='pixels', interactive=True)
        plt.connect('key_press_event', rs)
        plt.show()
        print("Draw a box by clicking and dragging on the image.")

# Function to process next image
def next_image():
    if image_queue:
        image_path = image_queue[0]
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        update_display(image, f"Inspecting: {os.path.basename(image_path)}")
    else:
        with output:
            clear_output(wait=True)
            print("All images processed!")

# Button callbacks
def on_save_clicked(b):
    global selected_region
    if selected_region and image_queue:
        image_path = image_queue.pop(0)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        x, y, w, h = selected_region
        cropped_image = image[y:y+h, x:x+w]  # Extract the selected region
        save_path = os.path.join(output_folder, f"cropped_{os.path.basename(image_path)}")
        cv2.imwrite(save_path, cropped_image)  # Save the cropped image
        print(f"Saved: {save_path}")
        selected_region = None  # Reset selected region
    next_image()

def on_skip_clicked(b):
    if image_queue:
        image_queue.pop(0)  # Skip the current image
    next_image()

# Create buttons
save_button = Button(description="Save Cropped", button_style='primary', layout=Layout(width='150px'))
skip_button = Button(description="Skip", button_style='warning', layout=Layout(width='150px'))

save_button.on_click(on_save_clicked)
skip_button.on_click(on_skip_clicked)

controls = HBox([save_button, skip_button])
display(VBox([output, controls]))

# Start processing
next_image()